In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
 

In [2]:
#import libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow.keras.backend as K
from keras.models import load_model

from sklearn.linear_model import LogisticRegression

# data reading

In [3]:
#image preproccessing
train = ImageDataGenerator(rescale =1./255)
test= ImageDataGenerator(rescale= 1/255)
valid = ImageDataGenerator(rescale =1./255)

In [4]:
#save data
photo_size = 256
traindata = train.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/train",target_size=(photo_size,photo_size) ,class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'] )
testdata= test.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/test", target_size=(photo_size,photo_size),class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'])
validdata =  valid.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/valid", target_size=(photo_size,photo_size),class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'])

Found 2536 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


# mobile  

In [6]:
feature_extractor_model = "https://tfhub.dev/sayannath/mobilevit_xxs_1k_256_fe/1"

pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(photo_size, photo_size, 3), trainable=False)

2022-12-29 23:11:39.256478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 23:11:39.346677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 23:11:39.347513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-29 23:11:39.348703: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(8, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid") 
    ])

model.compile(
  optimizer="adam",
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['acc'])

In [8]:
histroy_1 = model.fit(traindata, validation_data = validdata, epochs = 25 )

2022-12-29 23:11:51.390300: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-12-29 23:12:01.937827: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


80/80 [==============================] - 34s 228ms/step - loss: 0.5263 - acc: 0.7342 - val_loss: 0.4904 - val_acc: 0.7500
Epoch 2/25
80/80 [==============================] - 9s 115ms/step - loss: 0.3582 - acc: 0.8399 - val_loss: 0.5154 - val_acc: 0.7300
Epoch 3/25
80/80 [==============================] - 9s 117ms/step - loss: 0.2697 - acc: 0.8825 - val_loss: 0.5116 - val_acc: 0.7900
Epoch 4/25
80/80 [==============================] - 9s 106ms/step - loss: 0.2080 - acc: 0.9207 - val_loss: 0.6412 - val_acc: 0.7700
Epoch 5/25
80/80 [==============================] - 9s 114ms/step - loss: 0.1845 - acc: 0.9223 - val_loss: 0.5056 - val_acc: 0.8300
Epoch 6/25
80/80 [==============================] - 9s 111ms/step - loss: 0.1264 - acc: 0.9511 - val_loss: 0.6323 - val_acc: 0.7300
Epoch 7/25
80/80 [==============================] - 9s 115ms/step - loss: 0.1062 - acc: 0.9566 - val_loss: 0.7297 - val_acc: 0.7800
Epoch 8/25
80/80 [==============================] - 9s 107ms/step - loss: 0.0784 - acc

In [19]:
model.evaluate(testdata)

10/10 [==============================] - 1s 96ms/step - loss: 0.9184 - acc: 0.8400


[0.9183676242828369, 0.8399999737739563]

In [10]:
model.save("mobile.h5")

# inception

In [11]:
def create_inception_model():
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    base_model = InceptionV3(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    # for layer_idx in range(len(pretrained_model.layers)):
    #     if layer_idx not in [1,2,3,305,306,307,308,309,310]:
    #         pretrained_model.layers[layer_idx].trainable = False
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import layers
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(95, activation='softmax')(x)
    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model
inception_model=create_inception_model()


87924736/87910968 [==============================] - 0s 0us/step


In [12]:
incephist = inception_model.fit(traindata,validation_data=validdata, epochs=100)

Epoch 1/100
80/80 [==============================] - 14s 127ms/step - loss: 2.6634 - accuracy: 0.6266 - val_loss: 0.5841 - val_accuracy: 0.7700
Epoch 2/100
80/80 [==============================] - 9s 109ms/step - loss: 0.8583 - accuracy: 0.7468 - val_loss: 0.7609 - val_accuracy: 0.7000
Epoch 3/100
80/80 [==============================] - 8s 102ms/step - loss: 0.5665 - accuracy: 0.7906 - val_loss: 0.4927 - val_accuracy: 0.8000
Epoch 4/100
80/80 [==============================] - 8s 103ms/step - loss: 0.4325 - accuracy: 0.8131 - val_loss: 0.5574 - val_accuracy: 0.7600
Epoch 5/100
80/80 [==============================] - 8s 101ms/step - loss: 0.3732 - accuracy: 0.8435 - val_loss: 0.5964 - val_accuracy: 0.7400
Epoch 6/100
80/80 [==============================] - 9s 108ms/step - loss: 0.2667 - accuracy: 0.8876 - val_loss: 0.6463 - val_accuracy: 0.7800
Epoch 7/100
80/80 [==============================] - 8s 103ms/step - loss: 0.2478 - accuracy: 0.9018 - val_loss: 0.5304 - val_accuracy: 0.820

In [16]:
inception_model.evaluate(testdata)

10/10 [==============================] - 1s 105ms/step - loss: 1.8567 - accuracy: 0.8333


[1.856748104095459, 0.8333333134651184]

In [17]:
inception_model.save("inception.h5")

# vgg

In [38]:
def create_vgg_model():
    # load model without classifier layers
    model = VGG16(include_top=False, input_shape=(photo_size, photo_size, 3))
    """for layer_idx in range(len(model.layers)):
        if layer_idx not in [1,2,3,15,16,17,18]:
            model.layers[layer_idx].trainable = False"""
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(64, activation='relu')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model
vgg_model=create_vgg_model()
vgg_model.compile(optimizer=Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy()
, metrics=['accuracy'])

In [39]:
vgghist=vgg_model.fit(traindata,epochs=45,steps_per_epoch = 40 , validation_data=validdata)

Epoch 1/100
40/40 [==============================] - 12s 275ms/step - loss: 0.6799 - accuracy: 0.5805 - val_loss: 0.6140 - val_accuracy: 0.6900
Epoch 2/100
40/40 [==============================] - 11s 262ms/step - loss: 0.5552 - accuracy: 0.7150 - val_loss: 0.5519 - val_accuracy: 0.7500
Epoch 3/100
40/40 [==============================] - 11s 262ms/step - loss: 0.5622 - accuracy: 0.7229 - val_loss: 0.5588 - val_accuracy: 0.7100
Epoch 4/100
40/40 [==============================] - 11s 263ms/step - loss: 0.4841 - accuracy: 0.7787 - val_loss: 0.8601 - val_accuracy: 0.6200


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f7af0cfd9e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 5/100
40/40 [==============================] - 11s 263ms/step - loss: 0.4849 - accuracy: 0.7779 - val_loss: 0.5186 - val_accuracy: 0.7300
Epoch 6/100
40/40 [==============================] - 11s 262ms/step - loss: 0.4267 - accuracy: 0.8049 - val_loss: 0.5039 - val_accuracy: 0.7400
Epoch 7/100
40/40 [==============================] - 11s 266ms/step - loss: 0.4146 - accuracy: 0.8070 - val_loss: 0.4777 - val_accuracy: 0.7600
Epoch 8/100
40/40 [==============================] - 11s 263ms/step - loss: 0.3702 - accuracy: 0.8312 - val_loss: 0.5419 - val_accuracy: 0.7300


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f7af0cfd9e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 9/100
40/40 [==============================] - 11s 261ms/step - loss: 0.3337 - accuracy: 0.8559 - val_loss: 0.4781 - val_accuracy: 0.7700


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f7af0cfd9e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 10/100
40/40 [==============================] - 11s 267ms/step - loss: 0.3531 - accuracy: 0.8500 - val_loss: 0.5586 - val_accuracy: 0.7700
Epoch 11/100
40/40 [==============================] - 11s 263ms/step - loss: 0.3037 - accuracy: 0.8702 - val_loss: 0.6013 - val_accuracy: 0.7200


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f7af0cfd9e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 12/100
40/40 [==============================] - 11s 267ms/step - loss: 0.2759 - accuracy: 0.8836 - val_loss: 0.5546 - val_accuracy: 0.7300
Epoch 13/100
40/40 [==============================] - 11s 262ms/step - loss: 0.2676 - accuracy: 0.8949 - val_loss: 0.5201 - val_accuracy: 0.7900
Epoch 14/100
40/40 [==============================] - 11s 265ms/step - loss: 0.2378 - accuracy: 0.9000 - val_loss: 0.6458 - val_accuracy: 0.7500
Epoch 15/100
40/40 [==============================] - 11s 263ms/step - loss: 0.2553 - accuracy: 0.8893 - val_loss: 0.6318 - val_accuracy: 0.7100
Epoch 16/100
40/40 [==============================] - 11s 267ms/step - loss: 0.2389 - accuracy: 0.9008 - val_loss: 0.6117 - val_accuracy: 0.7400
Epoch 17/100
40/40 [==============================] - 11s 261ms/step - loss: 0.1674 - accuracy: 0.9355 - val_loss: 0.4628 - val_accuracy: 0.8100
Epoch 18/100
37/40 [==========================>...] - ETA: 0s - loss: 0.1494 - accuracy: 0.9400

KeyboardInterrupt: 

In [41]:
vgg_model.evaluate(testdata)

10/10 [==============================] - 3s 285ms/step - loss: 0.4302 - accuracy: 0.8467


[0.4302464723587036, 0.846666693687439]

In [5]:
vgg_model.save("vgg16_model")

NameError: name 'vgg_model' is not defined

# Stacking

In [16]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def acc(y_true, y_pred):
    tp = 0
    total = 0
    
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            tp+=1
        total+=1
    return (tp / total)*100

In [15]:
dependencies = {'KerasLayer':hub.KerasLayer,
    'acc': f1_m }

In [7]:
def read_data(list, photo_size = 256):  
    data = []
    labels = []
    count = 1
    for i in list:
        for filename in os.listdir(i):
            img  = cv2.imread(i + "//" +filename)
            img = cv2.resize(img,(photo_size,photo_size))
            data.append(img)
            labels.append(count) 
        count = 0
    return data,labels

train_add_autis = "/kaggle/input/autistic-children-facial-data-set/train/autistic"
train_add_nonautis = "/kaggle/input/autistic-children-facial-data-set/train/non_autistic"


x_train, y_train = read_data([train_add_autis, train_add_nonautis])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = x_train / 255

In [8]:
all_models = []
model = load_model("/kaggle/working/mobile.h5",custom_objects=dependencies)
all_models.append(model)

model = load_model("/kaggle/working/vgg16_model.h5",custom_objects=dependencies)
all_models.append(model)

In [9]:
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat #
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [10]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit the meta learner
    model = LogisticRegression() #meta learner
    model.fit(stackedX, inputy)
    return model

In [11]:
model = fit_stacked_model(all_models, x_train,y_train)

2022-12-30 01:57:56.960473: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.
2022-12-30 01:57:59.195556: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.
2022-12-30 01:58:00.778197: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-30 01:58:03.317581: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-12-30 01:58:14.804382: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.
2022-12-30 01:58:18.305985: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.


In [12]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [13]:
train_add_autis = "/kaggle/input/autistic-children-facial-data-set/test/autistic"
train_add_nonautis = "/kaggle/input/autistic-children-facial-data-set/test/non_autistic"


x_test, y_test = read_data([train_add_autis, train_add_nonautis])

x_test = np.array(x_test)
y_test = np.array(y_test)

x_test = x_test / 255

In [ ]:
yhat = stacked_prediction(all_models, model, x_test)
score = acc(y_test, yhat)
print(score)
